In [1]:
import pandas as pd
df = pd.read_csv("./data/e2e_table.csv")
display(df)

,dataset,type,acc,mean_s,sum_s,bandwidth_GB,mean_factor,bw_factor
0,CIFAR Large,linf_opt,0.6534,130.735711,5098.692711,102.76,6,25
1,CIFAR Large,plain,0.6644,20.971711,817.896711,4.19,1,1
2,CIFAR Small,l2,0.5415,281.909316,10994.463316,34.77,134,37
3,CIFAR Small,l2_opt,0.5429,52.876000,2115.040000,6.73,25,7
4,CIFAR Small,linf,0.5410,251.674158,9815.292158,23.34,120,25
5,CIFAR Small,linf_opt,0.5352,38.217026,1490.464026,23.34,18,25
6,CIFAR Small,plain,0.5384,2.105278,77.895278,0.95,1,1
7,MNIST,l2,0.9744,132.016077,5280.643077,10.75,39,37
8,MNIST,l2_opt,0.9713,26.019949,1040.797949,2.81,8,10
9,MNIST,linf,0.9737,123.719513,4948.780513,7.21,37,25


In [2]:
header = r"""
\begin{table}
    \centering
    \resizebox{\columnwidth}{!}{%
    \begin{tabular}{lrcrrrcrr}
        \toprule
             & Acc.               && \multicolumn{3}{c}{Computation Time} & & \multicolumn{2}{c}{Bandwidth} \\
             \cline{4-6}\cline{8-9}
        Type & [R40] && Round [s] & Total [m] & Factor & & Total [GB] & Factor \\"""


mnist_header = r"""        \toprule
        \multicolumn{9}{c}{\textsc{MNIST} (19k params, rsl 5k params)} \\"""

cifar_small_header = r"""        \toprule
        \multicolumn{9}{c}{\textsc{CIFAR-10 S} (62k params, rsl 12k params)} \\"""

cifar_large_header = r"""        \toprule
        \multicolumn{9}{c}{\textsc{CIFAR-10 L} (273k params)} \\"""

footer = r"""        \bottomrule
    \end{tabular}
    }
\end{table}
"""

type_order = {
    "plain": 0,
    "l2": 1,
    "l2_opt": 2,
    "linf": 3,
    "linf_opt": 4
}

df['type_order'] = df['type'].map(type_order)
df = df.sort_values(by=["dataset", "type_order"])

type_print = {
    "plain": r"plain",
    "l2": r"$L_2$",
    "l2_opt": r"$L_2^{(rsl)}$",
    "linf": r"$L_\infty$",
    "linf_opt": r"$L_\infty^{(p)}$"
}


def format_row(x):
    # total_time = f"{round(x.sum_s/60)}m" if round(x.sum_s/60) > 0 else f"{round(x.sum_s)}s"
    return fr"{type_print[x.type]} & {round(x.acc, 2)} & & {round(x.mean_s)} & {round(x.sum_s/60)} & {x.mean_factor}x & & {round(x.bandwidth_GB,1)} & {x.bw_factor}x \\"

# PRINT THE LATEX TABLE

print(header)

print(mnist_header)


for row in df[df["dataset"]=="MNIST"].itertuples():
    print("        " + format_row(row))

print(cifar_small_header)
    
for row in df[df["dataset"]=="CIFAR Small"].itertuples():
    print("        " + format_row(row))

print(cifar_large_header)
    
for row in df[df["dataset"]=="CIFAR Large"].itertuples():
    print("        " + format_row(row))

    
print(footer)




\begin{table}
    \centering
    \resizebox{\columnwidth}{!}{%
    \begin{tabular}{lrcrrrcrr}
        \toprule
             & Acc.               && \multicolumn{3}{c}{Computation Time} & & \multicolumn{2}{c}{Bandwidth} \\
             \cline{4-6}\cline{8-9}
        Type & [R40] && Round [s] & Total [m] & Factor & & Total [GB] & Factor \\
        \toprule
        \multicolumn{9}{c}{\textsc{MNIST} (19k params, rsl 5k params)} \\
        plain & 0.97 & & 3 & 2 & 1x & & 0.3 & 1x \\
        $L_2$ & 0.97 & & 132 & 88 & 39x & & 10.8 & 37x \\
        $L_2^{(rsl)}$ & 0.97 & & 26 & 17 & 8x & & 2.8 & 10x \\
        $L_\infty$ & 0.97 & & 124 & 82 & 37x & & 7.2 & 25x \\
        $L_\infty^{(p)}$ & 0.97 & & 20 & 13 & 6x & & 7.2 & 25x \\
        \toprule
        \multicolumn{9}{c}{\textsc{CIFAR-10 S} (62k params, rsl 12k params)} \\
        plain & 0.54 & & 2 & 1 & 1x & & 0.9 & 1x \\
        $L_2$ & 0.54 & & 282 & 183 & 134x & & 34.8 & 37x \\
        $L_2^{(rsl)}$ & 0.54 & & 53 & 35 & 25x & & 6.7 & 7